## Circuit cutting with automatic cut finding using the Circuit Knitting Toolbox

### Import relevant modules

In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Options, Session, Sampler

from circuit_knitting_toolbox.circuit_cutting import WireCutter

### Create a circuit to cut

In [2]:
qc = QuantumCircuit(5)
for i in range(5):
    qc.h(i)
qc.cx(0, 1)
for i in range(2, 5):
    qc.t(i)
qc.cx(0, 2)
qc.rx(np.pi / 2, 4)
qc.rx(np.pi / 2, 0)
qc.rx(np.pi / 2, 1)
qc.cx(2, 4)
qc.t(0)
qc.t(1)
qc.cx(2, 3)
qc.ry(np.pi / 2, 4)
for i in range(5):
    qc.h(i)

qc.draw()

┌───┐                ┌─────────┐   ┌───┐   ┌───┐
q_0: ┤ H ├──■───────■─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤┌─┴─┐     │     ├─────────┤   ├───┤   ├───┤
q_1: ┤ H ├┤ X ├─────┼─────┤ Rx(π/2) ├───┤ T ├───┤ H ├
     ├───┤├───┤   ┌─┴─┐   └─────────┘   └───┘   ├───┤
q_2: ┤ H ├┤ T ├───┤ X ├────────■──────────■─────┤ H ├
     ├───┤├───┤   └───┘        │        ┌─┴─┐   ├───┤
q_3: ┤ H ├┤ T ├────────────────┼────────┤ X ├───┤ H ├
     ├───┤├───┤┌─────────┐   ┌─┴─┐   ┌──┴───┴──┐├───┤
q_4: ┤ H ├┤ T ├┤ Rx(π/2) ├───┤ X ├───┤ Ry(π/2) ├┤ H ├
     └───┘└───┘└─────────┘   └───┘   └─────────┘└───┘

## Set up the Qiskit runtime session

In [3]:
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='<YOUR_INSTANCE>',
    token="<YOUR_API_TOKEN>",
)
session = Session(service=service, backend='backend_name')

### Find cuts that match our criteria and evaluate the subcircuits within a Qiskit Session context

In [5]:
options = Options(resilience_level=1, optimization_level=3, execution={'shots': 8192})
with Sampler(session=session, options=options) as sampler:
    cutter = WireCutter(qc, sampler)
    cuts = cutter.cut_automatic(
        max_subcircuit_width=3,
        max_subcircuit_cuts=10,
        max_subcircuit_size=12,
        max_cuts=10,
        num_subcircuits=[2],
    )
    subcircuit_instance_probabilities = cutter.evaluate(cuts)

/Users/caleb/opt/anaconda3/envs/ckt/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The BaseSampler.__enter__ method is deprecated as of Qiskit Terra 0.22.0 and will be removed no sooner than 3 months after the releasedate. BaseSampler should be initialized directly.
  


Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               300
Tried aggregator 3 times.
MIP Presolve eliminated 23 rows and 5 columns.
MIP Presolve modified 4 coefficients.
Aggregator did 22 substitutions.
Reduced MIP has 47 rows, 24 columns, and 142 nonzeros.
Reduced MIP has 15 binaries, 9 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.36 ticks)
Found incumbent of value 1.000000 after 0.01 sec. (0.51 ticks)
Probing fixed 15 vars, tightened 22 bounds.
Probing time = 0.00 sec. (0.03 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 47 rows and 24 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.02 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.56 ticks)
Parallel b&c, 16 threads:
  Real time             =    0.00 sec. (0

/Users/caleb/opt/anaconda3/envs/ckt/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The BaseSampler.__exit__ method is deprecated as of Qiskit Terra 0.22.0 and will be removed no sooner than 3 months after the releasedate. BaseSampler should be initialized directly.
  # This is added back by InteractiveShellApp.init_path()


## Recompose the circuit and verify the error between the full and cut circuit distributions is within tolerance

In [6]:
ordered_probabilities = cutter.recompose(subcircuit_instance_probabilities, cuts, num_threads=4)
metrics = cutter.verify(ordered_probabilities)
print(metrics)

{'nearest': {'chi2': 0.0011186143314989884, 'Mean Squared Error': 4.545574109408901e-06, 'Mean Absolute Percentage Error': 6.266595462888194, 'Cross Entropy': 2.6008050249835195, 'HOP': 0.8979776188036324}, 'naive': {'chi2': 0.0011186143314989886, 'Mean Squared Error': 4.5455741094088934e-06, 'Mean Absolute Percentage Error': 6.266595462888201, 'Cross Entropy': 2.6008050249835195, 'HOP': 0.8979776188036325}}
